In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib inline
plt.style.use('ggplot')
import seaborn as sns

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_csv('df2.csv', index_col=0)

In [4]:
zri = pd.read_csv('df2.csv', index_col=0)
zri['MortDeliq30-89'] = zri['MortDeliq30-89'].fillna(zri['MortDeliq30-89'].mean())
zri['MortDeliq90'] = zri['MortDeliq90'].fillna(zri['MortDeliq90'].mean())
zri['education'] = zri['high_school_diploma'] + zri['associates_degree'] * 2 + zri['bachelors_degree'] * 3 +zri['masters_degree']* 4
zri['AGIncome'] = (zri['AGIncome']*1000)/zri['total_pop']
zri['TotIncome'] = (zri['TotIncome']*1000)/zri['total_pop']
zri['SalariesWages'] = (zri['SalariesWages']*1000)/zri['total_pop']
zri['NumUnemply'] =(zri['NumUnemply']*1000)/zri['total_pop']
zri['TotTaxes'] = (zri['TotTaxes']*1000)/zri['total_pop']


In [5]:
dropped = [ 'City', 'State', 'Metro', 'CountyName', 'SizeRank',
       'Year-Month', 'value', 'Month', 'Year', 'year',  'geo_id', 'FIPSCode',
          'high_school_diploma', 'associates_degree', 'bachelors_degree',
       'masters_degree' , 'Employed', 'Unemployed',
          'AGIncome','TotIncome', 'SalariesWages', 'Labor\nForce']

In [6]:
train = pd.merge(zri.loc[zri['Year']==2015].rename(columns = {'value' : 'value_2015'}),
                 zri.loc[zri['Year']==2016][['ZIPCODE', 'Month','value']],
                 how = 'left', on = ['ZIPCODE', 'Month'])
test = pd.merge(zri.loc[zri['Year']==2018].rename(columns = {'value' : 'value_2018'}),
                zri.loc[zri['Year']==2019][['ZIPCODE', 'Month','value']],
                how = 'left', on = ['ZIPCODE', 'Month'])
train = train.dropna()
test = test.dropna()
X_train = train.drop(columns = dropped)
X_test = test.drop(columns = dropped)
y_train = train['value']
y_test = test['value']

# Tree Based Models

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(
#     X,
#     y,
#     test_size=0.3,
#     random_state=0)
# X_train.shape, X_test.shape

# Gradient Boost

In [7]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [8]:
gbm = GradientBoostingRegressor()

gbm.set_params(n_estimators = 1000, 
            learning_rate = .1, #.3,
            max_depth = 2, #2,
            min_samples_leaf = 3)

gbm.fit(X_train,y_train)

print('-'*50)
print("The train set R^2 is %.3f" %(gbm.score(X_train, y_train)))
print("The test set R^2 is %.3f" %(gbm.score(X_test,y_test)))

--------------------------------------------------
The train set R^2 is 0.994
The test set R^2 is 0.973


### Grid Search for Gradient Boost

In [9]:
# n_estimator: 

x = np.logspace(1, 3, 100)
x = x.astype(np.int64)

In [ ]:
param_grid={'n_estimators':x,
           'learning_rate': [.3,.2,0.1],
            'max_depth':[1,2,3],
            'min_samples_leaf':[3]}

grid_search = GridSearchCV(estimator= gbm, param_grid= param_grid)
%time grid_search.fit(X_train,y_train)

In [ ]:
print(grid_search.best_estimator_)
print(grid_search.best_params_)
print(grid_search.best_score_)

In [ ]:
gbm.get_params

### Gradient Boost - Feature Importance

In [ ]:
sorted(zip(X.columns, gbm.feature_importances_), key=lambda t:t[1], reverse=True)

In [ ]:
# creating a dataframe of the top 12 features in feature importance and graphing the features

d = {'Features': ['Overall Quality', 'GrLivArea', 'TotalBsmtSF','Neighborhood','Total Bathrooms','Curb Appeal', \
                 'YearBuilt','Fireplaces','LotArea','Overall Condition','HeatingQC','Central Air'], \
     'Scores': [0.4389, 0.1795,0.1071,0.0822,0.0589,0.0401,0.0175,0.0153,0.0149,0.0091,0.0052,0.0040]}
important_features = pd.DataFrame(data=d)

In [ ]:
important_features = important_features.rename(columns={'Scores': "Coefficients"})

In [ ]:
sns.barplot(x='Coefficients', y='Features', data=important_features)

### Gradient Boost - Plotting Residuals

In [ ]:
# plotting the actual values vs. the predicted values

gbm_predicted = pd.DataFrame(gbm.predict(X))
gbm_predicted = gbm_predicted.rename(columns={0: "Predicted"})
gbm_predicted_actual = pd.concat([gbm_predicted, g], axis=1, sort=False)
sns.scatterplot(x=gbm_predicted_actual['SalePrice'], y=gbm_predicted_actual['Predicted'], data=gbm_predicted_actual)

In [ ]:
# plotting the actual values vs. absolute residuals 

gbm_residuals = pd.DataFrame(y - gbm.predict(X))

In [ ]:
gbm_residuals = gbm_residuals.rename(columns={"SalePrice": "Residual Values"})

In [ ]:
f = clean_df[['SalePrice']]

In [ ]:
gbm_resid = pd.concat([gbm_residuals, f], axis=1, sort=False)
sns.scatterplot(x=gbm_resid['SalePrice'], y=gbm_resid['Residual Values'], data=gbm_resid).set_title \
('Gradient Boost Residuals Plot')

# XG Boost

In [4]:
from xgboost import XGBRegressor
from sklearn.metrics import accuracy_score

In [5]:
model1 = XGBRegressor()
model1.fit(X_train, y_train)

NameError: name 'X_train' is not defined

In [ ]:
print("The train set R^2 is %.3f" %(model1.score(X_train, y_train)))
print("The test set R^2 is %.3f" %(model1.score(X_test,y_test)))

In [ ]:
model1.get_booster().get_score(importance_type="gain")

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from pprint import pprint

In [ ]:
rf = RandomForestRegressor(random_state = 999)

In [ ]:
from sklearn import ensemble

randomForest1 = ensemble.RandomForestRegressor()
bagging = ensemble.BaggingClassifier()

randomForest1.set_params(random_state=42, n_estimators=464, max_features=4, min_samples_split=8)

print(randomForest1.fit(X_train,y_train))

print(randomForest1.score(X_train,y_train))
randomForest1.score(X_test,y_test)

### Random Forest - Residuals Plot

In [ ]:
residuals = pd.DataFrame(y - randomForest1.predict(X))

In [ ]:
predicted = pd.DataFrame(randomForest1.predict(X))

In [ ]:
predicted = predicted.rename(columns={0: "Predicted"})

In [ ]:
residuals = residuals.rename(columns={"SalePrice": "Residual Values"})

In [ ]:
g = clean_df[['SalePrice']]

In [ ]:
resid_actual = pd.concat([residuals, g], axis=1, sort=False)
resid_actual

In [ ]:
# absolute residual values vs sale price graph

sns.scatterplot(x=resid_actual['SalePrice'], y=resid_actual['Residual Values'], data=resid_actual).set_title \
('Random Forest Residuals Plot')

In [ ]:
predicted_actual = pd.concat([predicted, g], axis=1, sort=False)

In [ ]:
# predicted vs. actual sale price values plot 

sns.scatterplot(x=predicted_actual['SalePrice'], y=predicted_actual['Predicted'], data=predicted_actual)

# Graphviz - Getting Image of Decision Tree from Random Forest

In [ ]:
from sklearn import tree

In [ ]:
from sklearn.tree import export_graphviz

In [ ]:
estimator = randomForest1.estimators_[5]

In [ ]:
# Export as dot file
export_graphviz(estimator, out_file='tree.dot', 
                feature_names = X.columns,
                class_names = 'SalePrice',
                rounded = True, proportion = False, 
                precision = 2, filled = True)

In [ ]:
from subprocess import call
call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=600'])

# Extra Trees Classifier

In [ ]:
import pandas as pd
import numpy as np

from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
model2 = ExtraTreesClassifier()
model2.fit(X,y)
print(model2.feature_importances_) #use inbuilt class feature_importances of tree based classifiers
#plot graph of feature importances for better visualization
feat_importances = pd.Series(model2.feature_importances_, index=X.columns)
feat_importances.nlargest(55).plot(kind='barh', figsize=(6,8))
plt.show()

In [ ]:
clean_df.to_pickle('./data/clean_df.pkl')